# Explanatory Data Visualization
**Presentation Visualizations and Dashboards**

Note: goal is to show ease of itegrating RAPIDS and GPUS into production dashboard presentations

## Overview and Requirements
Super short version of intro notebook and restate requirments

## Import

## Summary of interesting analytics findings
Summary of what we want to show, some sketches of how we want to show it, and the type of audiene we are addressing, and what context they will be shown


In [ ]:
## 